# Downloading, converting, and subsetting Forest Service data
* Author: Katie Murenbeeld
* Date: 19 Nov 2019
* Updated: 09 Dec 2019
* Updated: 23 May 2020, for use in a survival analysis
* Updated: 24 June 2020, needed to fix the fillna for incomplete activities and projects
* Updated: 09 Sept 2020, change in desired columns and time range (2005-2018) for use in survival analysis
* Updated: 17 Sept 2020, updated timber harvest and added hazardous fuels treatment datasets. Column header names have changed since July 2019. This notebook now reflects those changes. Remeber list(df) will show unique column headers.
* Updated 26 Sept 2020, only need planned through 2017 due to an issue with NEPA data for the reforestation and timber stand improvement datasets.
* Updated 18 Oct 2020, TSI and RF datasets have been updated. Cleaning up code. Take out step 5, reclassifying the activities. 

## Introduction:
Within this notebook I will show you:
1. Where to retrieve and download Forest Service activity data 
2. How to convert the data to a .csv file
3. How to "subset" the data for the desired state and information
4. How to convert the planned and completed dates to serial dates
5. How to calculate Time Lag (the difference between completed and planned date) and the difference in planned and completed area treated and store in a new column
6. How to create a new column "Completed" and fill with binary data, where 0 = FALSE (i.e. not completed) and 1 = TRUE (i.e. completed).
7. Write out a .csv file with the modified dataset


### 1. Retrieve and download Forest Service activity data
Forest Service activity data can be found at here: https://data.fs.usda.gov/geodata/edw/datasets.php

Please chose the dataset you would like to work with (in this notebook I will be using the Timber Harvest data) by clicking on the shapefile link for the desired dataset (insert screen shot here). Clicking on the link will automatically start the download. Once download is complete, move the dataset to your desired location. 

* Note: The metadata for the file is also useful and will define the different code names and numbers.

After you move the downloaded data, set the directories for your raw data and for the resulting .csv files.

In [1]:
# Set directories

raw_data_dir = '/Users/kathrynmurenbeeld/CODING/FOREST-TREATMENTS/DATA/FOREST-TREATMENTS/RAW/'
mod_data_dir = '/Users/kathrynmurenbeeld/CODING/FOREST-TREATMENTS/DATA/FOREST-TREATMENTS/MOD/'

### 2. Convert the data to a .csv file
Remember, the file you downloaded is a shape file. There are several files that comprise a GIS/ESRI shapefile. One of these is a .dbf. The .dbf is the file that will be converted to a .csv file. The conversion will produce a rather large .csv that should be stored in your modified data directory (mod_data_dir). To convert the data you will need a specific library called _simpledbf_ and the tool _Dbf5_ from _simpledbf_.

Before starting, I recommend that you rename the .dbf to reflect when you downloaded the raw data.

In [2]:
# Download the relevant libraries
import simpledbf 
from simpledbf import Dbf5 

# Other important libraries
import pandas as pd #For working with dataframes
import numpy as np #For working with arrays
import datetime as dt #For converting dates to serial datetimes


In [9]:
# Convert the most up to date timber harvest (th) data from dbf to csv
# It is useful to use a name that reflects the general time that you
# downloaded the data.

dbf_th = Dbf5(raw_data_dir + 'S_USA.Activity_TH_20201018.dbf', codec='utf-8') # Import the dbf and assign the code 

dbf_th.to_csv(mod_data_dir + 'USA_TH_20201018.csv') # Covert to csv

In [10]:
# Hazardous fuels (hf) treatments
dbf_hf = Dbf5(raw_data_dir + 'S_USA.Activity_HF_20201018.dbf', codec='utf-8') # Import the dbf and assign the code 

dbf_hf.to_csv(mod_data_dir + 'USA_HF_20201018.csv') # Covert to csv

In [8]:
# Reforestation (rf) treatments
dbf_rf = Dbf5(raw_data_dir + 'S_USA.Activity_RF_20201018.dbf', codec='utf-8') # Import the dbf and assign the code 

dbf_rf.to_csv(mod_data_dir + 'USA_RF_20201018.csv') # Covert to csv

In [3]:
# Timber Stand Improvement (tsi) treatments
dbf_tsi = Dbf5(raw_data_dir + 'S_USA.Activity_TSI_20201018.dbf', codec='utf-8')

dbf_tsi.to_csv(mod_data_dir + 'USA_TSI_20201018.csv')

In [331]:
# Use pandas to create dataframe from new csv
df = pd.read_csv(mod_data_dir + 'USA_TSI_20201018.csv').reset_index()
df.head(1) # Show the first row of the dataframe as a check.

/Users/kathrynmurenbeeld/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8,9,10,39,51,53,56,58,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,index,REGION_COD,ADMIN_FORE,ADMIN_FO_1,PROCLAIMED,ADMIN_DIST,DISTRICT_C,HOME_ORG,ACTIVITY_U,SUID,...,GIS_ACRES,EDW_INSERT,ETL_MODIFI,ETL_MODI_1,REV_DATE,UK_TABULAR,UK,DATA_SOU_1,SHAPE_AREA,SHAPE_LEN
0,0,2,2,Bighorn National Forest,202,Powder River Ranger District,1,20201,20201,0202011006160002000,...,151.693,2019-10-04,Y,NaN,NaN,AC420801045210452_0202011006160002000,0202_747_ACTP,0.0,0.000069,0.054128


In [332]:
# Check to see that the NEPA data is there
list(df)
#df['NEPA_DOC_N'].unique()
#df['NEPA_PROJE'].unique()

['index',
 'REGION_COD',
 'ADMIN_FORE',
 'ADMIN_FO_1',
 'PROCLAIMED',
 'ADMIN_DIST',
 'DISTRICT_C',
 'HOME_ORG',
 'ACTIVITY_U',
 'SUID',
 'FACTS_ID',
 'SUBUNIT',
 'SALE_NAME',
 'ACTIVITY_C',
 'ACTIVITY_N',
 'ACTIVITY_1',
 'NBR_UNITS_',
 'UOM',
 'NBR_UNITS1',
 'DATE_PLANN',
 'DATE_AWARD',
 'DATE_COMPL',
 'FY_PLANNED',
 'FY_AWARDED',
 'FY_COMPLET',
 'FUND_CODES',
 'COST_PER_U',
 'NEPA_PROJE',
 'NEPA_DOC_N',
 'NEPA_PRO_1',
 'METHOD_COD',
 'METHOD_DES',
 'EQUIPMENT_',
 'EQUIPMENT1',
 'IMPLEMENTA',
 'IMPLEMEN_1',
 'IMPLEMEN_2',
 'WORK_AGENT',
 'PRODUCTIVI',
 'PRODUCTI_1',
 'LAND_SUITA',
 'LAND_SUI_1',
 'OWNERSHIP_',
 'OWNERSHIP1',
 'ASPECT',
 'ELEVATION',
 'SLOPE',
 'STATE_ABBR',
 'SUBUNIT_NA',
 'TREATMENT_',
 'WATERSHED_',
 'SUBUNIT_CN',
 'SUBUNIT_SI',
 'SUBUNIT_UO',
 'STAGE',
 'STAGE_DESC',
 'DATA_SOURC',
 'ACCURACY',
 'CRC_VALUE',
 'GIS_ACRES',
 'EDW_INSERT',
 'ETL_MODIFI',
 'ETL_MODI_1',
 'REV_DATE',
 'UK_TABULAR',
 'UK',
 'DATA_SOU_1',
 'SHAPE_AREA',
 'SHAPE_LEN']

### 3. Subset the data for the desired area (state, region, or forest) of interest.
As you can see, there are many columns with this data. Review the metadata to decide which columns will be most useful. In this case, I only want to look at columns:
1. STATE_ABBR, the state
2. ADMIN_REGI, the administrative region, (REGION_COD for RF and TSI datasets)
3. ADMIN_FORE, the number for associated with a National Forest (sometimes the name is missing from the dataset)
4. ADMIN_FO_1, the name of the National Forest
5. NEPA_DOC_N, the name of a Forest Service NEPA project, which are comprised of different activities.
6. NEPA_PROJE, the NEPA project number
7. ACTIVITY_N, the name of the activity
8. ACTIVITY_2, the numerical activity code (may be ACTIVITY_C in other datasets)
9. NBR_UNITS_, the number of acres planned to be treated by an activity
10. NBR_UNITS1, the number of acres actually treated by an activity
11. DATE_PLANN, the date an activity was planned to be completed
12. DATE_COMPL, the date an activity was actually completed
 

In [333]:
# Create a new dataframe the columns from the dataframe
df2 = df[['STATE_ABBR','REGION_COD','ADMIN_FORE', 'NEPA_DOC_N', 'NEPA_PROJE','ACTIVITY_N',
         'ACTIVITY_C','NBR_UNITS_','NBR_UNITS1','DATE_PLANN','DATE_COMPL']].copy()

# .copy() is important...

df2.head(1) # Check that the correct columns are in the new dataframe

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL
0,WY,2,2,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,19.0,19.0,2004-09-13,NaN


Finally, create a new data frame with only the data for specific administrative regions. In this case Regions 1 through 6. Use the .loc function in pandas to essentially slice the data by state.

In [334]:
# Use .loc and slice where column ADMIN_REGI (REGION_COD for rf and tsi)is equal to 1.
#df_r01 = df2.loc[df2['ADMIN_REGI']==1].copy()
df_r01 = df2.loc[df2['REGION_COD']==1].copy()
df_r01.head(5) # Check the new dataframe

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL
1559,ID,1,4,DEFAULT FOR NOT REQUIRED,NOT REQD,Fertilization,4550,40.0,40.0,2020-05-28,2020-05-28
1560,ID,1,4,DEFAULT FOR NOT REQUIRED,NOT REQD,Fertilization,4550,35.0,35.0,2020-05-26,2020-05-26
1676,MT,1,3,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,16.0,16.0,1976-10-01,1976-10-01
1677,MT,1,3,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,30.0,30.0,1967-06-01,1967-06-01
1678,MT,1,3,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,19.0,19.0,1967-07-01,1967-07-01


In [335]:
# Use .loc and slice where column ADMIN_REGI (REGION_COD)is equal to 1.

#df_r02 = df2.loc[df2['ADMIN_REGI']==2].copy()
df_r02 = df2.loc[df2['REGION_COD']==2].copy()
df_r02.head(5) # Check the new dataframe

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL
0,WY,2,2,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,19.0,19.0,2004-09-13,NaN
1,WY,2,2,(PALS)FORESTWIDE WILDLAND-URBAN INTERFACE FUEL...,30199,Precommercial Thin,4521,14.0,14.0,2016-07-22,2016-09-30
2,WY,2,2,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,31.0,31.0,1969-09-01,1969-09-01
3,WY,2,2,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,3.0,3.0,1997-07-30,NaN
4,WY,2,2,DEFAULT FOR NOT REQUIRED,NOT REQD,Tree Release and Weed,4511,3.0,3.0,1981-09-30,1981-09-30


In [336]:
# Use .loc and slice where column ADMIN_REGI (REGION_COD)is equal to 1.

#df_r03 = df2.loc[df2['ADMIN_REGI']==3].copy()
df_r03 = df2.loc[df2['REGION_COD']==3].copy()
df_r03.head(5) # Check the new dataframe

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL
23646,AZ,3,1,NEPA Pending,NEPA PEND,Precommercial Thin,4521,377.0,377.0,2019-07-30,NaN
23750,AZ,3,1,(PALS)YARROW REFORESTATION,55913,Precommercial Thin,4521,310.0,310.0,2019-07-30,2020-09-02
23859,AZ,3,1,(PALS)LARSON FOREST RESTORATION PROJECT,42876,Precommercial Thin,4521,62.0,62.0,2019-10-01,NaN
59300,NM,3,2,(PALS)BIGHORN/STATELINE WILDLIFE HABITAT AND T...,42403,Precommercial Thin,4521,9.0,9.0,2016-10-01,2017-08-09
59301,NM,3,2,(PALS)LA JARA FUELS REDUCTION PROJECT,10419,Precommercial Thin,4521,36.0,36.0,2012-09-28,2012-09-28


In [337]:
# Use .loc and slice where column ADMIN_REGI is equal to 4.

#df_r04 = df2.loc[df2['ADMIN_REGI']==4].copy()
df_r04 = df2.loc[df2['REGION_COD']==4].copy()
df_r04.head(5) # Check the new dataframe

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL
24040,ID,4,2,(PALS)HIGH VALLEY INTEGRATED RESTORATION PROJECT,45790,Tree Release and Weed,4511,2.0,2.0,2020-05-08,2020-08-06
24128,ID,4,2,(PALS)HIGH VALLEY INTEGRATED RESTORATION PROJECT,45790,Tree Release and Weed,4511,4.0,4.0,2020-05-01,2020-08-06
25061,ID,4,2,(PALS)HIGH VALLEY INTEGRATED RESTORATION PROJECT,45790,Tree Release and Weed,4511,27.0,27.0,2020-06-05,2020-08-06
25074,ID,4,2,(PALS)HIGH VALLEY INTEGRATED RESTORATION PROJECT,45790,Tree Release and Weed,4511,4.0,4.0,2020-05-01,2020-08-06
25177,ID,4,2,(PALS)WILLIAMS CREEK PROJECT DECISION MEMO,45705,Precommercial Thin,4521,11.0,11.0,2015-09-28,NaN


In [338]:
# Use .loc and slice where column ADMIN_REGI is equal to 5.

#df_r05 = df2.loc[df2['ADMIN_REGI']==5].copy()
df_r05 = df2.loc[df2['REGION_COD']==5].copy()
df_r05.head(5) # Check the new dataframe

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL
5760,CA,5,5,WINDY GAP,27243,Precommercial Thin,4521,1.0,1.0,1987-05-15,1987-05-15
5761,CA,5,5,JULY,27299,Tree Release and Weed,4511,22.0,22.0,1992-08-15,1992-08-15
9683,OR,5,5,(PALS)MT. ASHLAND LATE SUCCESSIONAL RESERVE HA...,5048,Precommercial Thin,4521,55.0,55.0,2009-09-25,2010-12-01
9727,OR,5,5,(PALS)MT. ASHLAND LATE SUCCESSIONAL RESERVE HA...,5048,Precommercial Thin,4521,5.0,5.0,2009-09-25,2015-09-21
9958,CA,5,5,MCDOWELL,27312,Tree Release and Weed,4511,16.0,16.0,1988-09-15,1988-09-15


In [339]:
# Use .loc and slice where column ADMIN_REGI is equal to 6.

#df_r06 = df2.loc[df2['ADMIN_REGI']==6].copy()
df_r06 = df2.loc[df2['REGION_COD']==6].copy()
df_r06.head(5) # Check the new dataframe

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL
15421,OR,6,14,(PALS)GLASS RESTORATION PROJECT,50471,Precommercial Thin,4521,2.0,2.0,2020-05-02,NaN
15469,OR,6,14,(PALS)GLASS RESTORATION PROJECT,50471,Precommercial Thin,4521,6.0,6.0,2020-05-02,NaN
15470,OR,6,14,(PALS)GLASS RESTORATION PROJECT,50471,Precommercial Thin,4521,6.0,6.0,2020-05-02,NaN
15471,OR,6,14,(PALS)GLASS RESTORATION PROJECT,50471,Precommercial Thin,4521,4.0,4.0,2020-05-02,NaN
15509,OR,6,14,(PALS)GLASS RESTORATION PROJECT,50471,Precommercial Thin,4521,8.0,8.0,2020-05-02,NaN


### 4. Convert the planned and completed dates to serial dates.
In order to analyze this data through time, the current date data must be converted into serial dates using the datetime library. A new column will be added to the data frame for the new serial dates.

In [410]:
# Convert DATE_* columns to datetime. Add a new column for the serial dates

# Create a numpy array of all the Date Planned values
r03_datearrP = df_r03['DATE_PLANN'].values 
# Convert the values in that array to a serial datetime and name as a new variable
r03_datearr_dtP = pd.to_datetime(r03_datearrP) 
# Add a new column to the dataframe of the converted datetime
df_r03['SerDatesPlan'] = r03_datearr_dtP 

# Repeat for the Date Completed

# Create a numpy array of all the Date Completed values
r03_datearrC = df_r03['DATE_COMPL'].values
r03_datearr_dtC = pd.to_datetime(r03_datearrC)
df_r03['SerDatesComp'] = r03_datearr_dtC

df_r03.head(1) # Check to make sure the new columns show up

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL,SerDatesPlan,SerDatesComp
23646,AZ,3,1,NEPA Pending,NEPA PEND,Precommercial Thin,4521,377.0,377.0,2019-07-30,NaN,2019-07-30,NaT


Other useful columns to add include:
1. A column for year planned ('YP')
2. A column for year completed ('YC')

This will be useful for looking at the total number of times a treatment occurred through time and the total area completed through time. By looking at yearly totals, the resulting figures will look better.

In [411]:
# Use datetime to retrieve the year values.

df_r03['YP'] = df_r03['SerDatesPlan'].dt.year
df_r03['YC'] = df_r03['SerDatesComp'].dt.year

# Check that the two new columns showed up and are correct.
df_r03.head(1)

# Note, the results may need to be an integer not a float and vice versa. 
# Some activities have not been completed, so YC will have some Nan values
# which makes the entire YC column data floats. Due to the Nans you may recieve an error
# in which case Nans may need to go to 0 and then be converted to float or integer.

# If need be, convert floats to integers.
# Check the data type using pandas .dtype
df_r03['YP'].dtype

dtype('int64')

In [412]:
# Replace Nan with 0 if desired for YC. This will help with indexing the correct years while keeping Nans in the next code block.

df_r03['YC'] = df_r03['YC'].fillna(0)

# Convert float to integer
df_r03['YC'] = df_r03['YC'].astype('int64')

# Check data type again
df_r03['YC'].dtype

dtype('int64')

For this analysis, do I only want projects completed by end of year 2018? I think I want all project planned up to end of year 2018. I want to have the option to have censored data, but I think I would need to specify when they were completed? If they had been completed since 2018?

In [413]:
# Here I will sort out the data so that only items planned to be completed up to end of year 2018 are included.  

## I want projects planned through 2018, completed through 2018, and I want to include Nans in the SerDatesComp column.

mask_range = (df_r03['SerDatesPlan'] >= '01-01-1900') & (df_r03['SerDatesPlan'] <= '2018-12-31')
df_r03 = df_r03.loc[mask_range].copy()
df_r03 = df_r03.loc[df_r03['YC'] <= 2018].copy()
df_r03.head(10) # Check the data

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL,SerDatesPlan,SerDatesComp,YP,YC
59300,NM,3,2,(PALS)BIGHORN/STATELINE WILDLIFE HABITAT AND T...,42403,Precommercial Thin,4521,9.0,9.0,2016-10-01,2017-08-09,2016-10-01,2017-08-09,2016,2017
59301,NM,3,2,(PALS)LA JARA FUELS REDUCTION PROJECT,10419,Precommercial Thin,4521,36.0,36.0,2012-09-28,2012-09-28,2012-09-28,2012-09-28,2012,2012
59302,NM,3,2,(PALS)CHAMISAL ECOSYSTEM RESTORATION PROJECT,17105,Precommercial Thin,4521,4.0,4.0,2011-08-01,2011-09-01,2011-08-01,2011-09-01,2011,2011
59307,AZ,3,1,(PALS)NUTRIOSO WILDLAND URBAN INTERFACE FUEL R...,5263,Precommercial Thin,4521,34.0,34.0,2010-03-15,2011-05-31,2010-03-15,2011-05-31,2010,2011
59308,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,114.0,114.0,1996-10-01,1996-10-01,1996-10-01,1996-10-01,1996,1996
59309,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,15.0,15.0,1991-07-01,1991-07-01,1991-07-01,1991-07-01,1991,1991
59310,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,183.0,183.0,1999-08-03,1999-08-03,1999-08-03,1999-08-03,1999,1999
59311,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,183.0,183.0,1975-08-01,1975-08-01,1975-08-01,1975-08-01,1975,1975
59312,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,32.0,32.0,1999-09-30,1999-09-30,1999-09-30,1999-09-30,1999,1999
59313,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,8.0,8.0,1998-08-26,1999-02-01,1998-08-26,1999-02-01,1998,1999


In [416]:
df_r03.isna().any()
df_r03['SerDatesComp'].max()

Timestamp('2018-12-21 00:00:00')

### 5. Calculate the _time lag_ from planned to completed and the difference in area treated from planned to completed.
Use simple subtraction to calculated _time lag_ and the difference in area. Add these new values as new columns to the dataframe.

In [417]:
df_r03['TIME_LAG'] = (df_r03['SerDatesComp'] - df_r03['SerDatesPlan']).dt.days
df_r03.head(1) # check the new column

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL,SerDatesPlan,SerDatesComp,YP,YC,TIME_LAG
59300,NM,3,2,(PALS)BIGHORN/STATELINE WILDLIFE HABITAT AND T...,42403,Precommercial Thin,4521,9.0,9.0,2016-10-01,2017-08-09,2016-10-01,2017-08-09,2016,2017,312.0


In [418]:
df_r03['NBR_UNITS1'].dtype

dtype('float64')

In [419]:
## Make sure NBR_UNITS_ and NBR_UNITS1 are numeric

df_r03['NBR_UNITS1'] = pd.to_numeric(df_r03['NBR_UNITS1'])
df_r03['NBR_UNITS_'] = pd.to_numeric(df_r03['NBR_UNITS_'])
df_r03['NBR_UNITS1'].dtype

dtype('float64')

In [420]:
df_r03['AREA_DIFF'] = (df_r03['NBR_UNITS1'] - df_r03['NBR_UNITS_'])
df_r03.head(2) # check the new column

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL,SerDatesPlan,SerDatesComp,YP,YC,TIME_LAG,AREA_DIFF
59300,NM,3,2,(PALS)BIGHORN/STATELINE WILDLIFE HABITAT AND T...,42403,Precommercial Thin,4521,9.0,9.0,2016-10-01,2017-08-09,2016-10-01,2017-08-09,2016,2017,312.0,0.0
59301,NM,3,2,(PALS)LA JARA FUELS REDUCTION PROJECT,10419,Precommercial Thin,4521,36.0,36.0,2012-09-28,2012-09-28,2012-09-28,2012-09-28,2012,2012,0.0,0.0


### 6. Create a column named Completed and fill with 0 if activity has not been completed or 1 if the activity has been completed. Create a column named NEPA and fill with 0 if activity did not require NEPA or 1 if the activity did require NEPA.

Not sure if this is really needed at this point. May be easier to do in R for the survival analysis?

In [421]:
conditions2 = [ (df_r03['YC'] == 0), (df_r03['YC'] > 0)]

choices2 = [0, 1]

conditions3 = [ (df_r03['NEPA_DOC_N'] == "DEFAULT FOR NOT REQUIRED"), (df_r03['NEPA_DOC_N'] != "DEFAULT FOR NOT REQUIRED")]

choices3 = [0, 1]

df_r03['Completed'] = np.select(conditions2, choices2, default='none')

df_r03['NEPA'] = np.select(conditions3, choices3, default='none')

df_r03.head(10)

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL,SerDatesPlan,SerDatesComp,YP,YC,TIME_LAG,AREA_DIFF,Completed,NEPA
59300,NM,3,2,(PALS)BIGHORN/STATELINE WILDLIFE HABITAT AND T...,42403,Precommercial Thin,4521,9.0,9.0,2016-10-01,2017-08-09,2016-10-01,2017-08-09,2016,2017,312.0,0.0,1,1
59301,NM,3,2,(PALS)LA JARA FUELS REDUCTION PROJECT,10419,Precommercial Thin,4521,36.0,36.0,2012-09-28,2012-09-28,2012-09-28,2012-09-28,2012,2012,0.0,0.0,1,1
59302,NM,3,2,(PALS)CHAMISAL ECOSYSTEM RESTORATION PROJECT,17105,Precommercial Thin,4521,4.0,4.0,2011-08-01,2011-09-01,2011-08-01,2011-09-01,2011,2011,31.0,0.0,1,1
59307,AZ,3,1,(PALS)NUTRIOSO WILDLAND URBAN INTERFACE FUEL R...,5263,Precommercial Thin,4521,34.0,34.0,2010-03-15,2011-05-31,2010-03-15,2011-05-31,2010,2011,442.0,0.0,1,1
59308,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,114.0,114.0,1996-10-01,1996-10-01,1996-10-01,1996-10-01,1996,1996,0.0,0.0,1,0
59309,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,15.0,15.0,1991-07-01,1991-07-01,1991-07-01,1991-07-01,1991,1991,0.0,0.0,1,0
59310,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,183.0,183.0,1999-08-03,1999-08-03,1999-08-03,1999-08-03,1999,1999,0.0,0.0,1,0
59311,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,183.0,183.0,1975-08-01,1975-08-01,1975-08-01,1975-08-01,1975,1975,0.0,0.0,1,0
59312,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,32.0,32.0,1999-09-30,1999-09-30,1999-09-30,1999-09-30,1999,1999,0.0,0.0,1,0
59313,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,8.0,8.0,1998-08-26,1999-02-01,1998-08-26,1999-02-01,1998,1999,159.0,0.0,1,0


In [422]:
# Optional: drop the DATE_PLANN and DATE_COMPL columns to clean table up more.
#df_id = df_id.drop(['DATE_PLANN', 'DATE_COMP', 'YP', 'YC'], axis=1)
df_r03.head(20) # Check that the columns were dropped

,STATE_ABBR,REGION_COD,ADMIN_FORE,NEPA_DOC_N,NEPA_PROJE,ACTIVITY_N,ACTIVITY_C,NBR_UNITS_,NBR_UNITS1,DATE_PLANN,DATE_COMPL,SerDatesPlan,SerDatesComp,YP,YC,TIME_LAG,AREA_DIFF,Completed,NEPA
59300,NM,3,2,(PALS)BIGHORN/STATELINE WILDLIFE HABITAT AND T...,42403,Precommercial Thin,4521,9.0,9.0,2016-10-01,2017-08-09,2016-10-01,2017-08-09,2016,2017,312.0,0.0,1,1
59301,NM,3,2,(PALS)LA JARA FUELS REDUCTION PROJECT,10419,Precommercial Thin,4521,36.0,36.0,2012-09-28,2012-09-28,2012-09-28,2012-09-28,2012,2012,0.0,0.0,1,1
59302,NM,3,2,(PALS)CHAMISAL ECOSYSTEM RESTORATION PROJECT,17105,Precommercial Thin,4521,4.0,4.0,2011-08-01,2011-09-01,2011-08-01,2011-09-01,2011,2011,31.0,0.0,1,1
59307,AZ,3,1,(PALS)NUTRIOSO WILDLAND URBAN INTERFACE FUEL R...,5263,Precommercial Thin,4521,34.0,34.0,2010-03-15,2011-05-31,2010-03-15,2011-05-31,2010,2011,442.0,0.0,1,1
59308,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,114.0,114.0,1996-10-01,1996-10-01,1996-10-01,1996-10-01,1996,1996,0.0,0.0,1,0
59309,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,15.0,15.0,1991-07-01,1991-07-01,1991-07-01,1991-07-01,1991,1991,0.0,0.0,1,0
59310,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,183.0,183.0,1999-08-03,1999-08-03,1999-08-03,1999-08-03,1999,1999,0.0,0.0,1,0
59311,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,183.0,183.0,1975-08-01,1975-08-01,1975-08-01,1975-08-01,1975,1975,0.0,0.0,1,0
59312,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,32.0,32.0,1999-09-30,1999-09-30,1999-09-30,1999-09-30,1999,1999,0.0,0.0,1,0
59313,AZ,3,1,DEFAULT FOR NOT REQUIRED,NOT REQD,Precommercial Thin,4521,8.0,8.0,1998-08-26,1999-02-01,1998-08-26,1999-02-01,1998,1999,159.0,0.0,1,0


### 7. Write to .csv file
Write the final version of the data frame to a .csv file and store in the mod_data_dir. Name in such a way that you will know this is for the correct activity type (timber harvest, timber stand improvement, etc.) in Idaho.

In [423]:
# Use pandas .to_csv to convert a dataframe to a .csv file. 
# The default encoding is utf-8, but this can be adjusted if desired.

df_r03.to_csv(mod_data_dir + 'ID_TSI_c20201019_survana_reg03.csv')

Repeat the above steps from Section 4 for the other regions and other activity datasets. 